In [2]:
import pandas as pd
import numpy as np
import math
from qiskit import QuantumCircuit
from qiskit import Aer, execute
import os

### Lectura de archivos de datos
1. Se importa como **'data'** el archivo de datos proporcionado. Los datos contienen los pedidos Train y Test a comparar. Se compararan todos los pedidos Train con los Test.
   - Se descartan las columnas que no se utilizaran en la comparación.

&nbsp;
2. Se importa como **'data denominacion'** el archivo que contiene el nombre real de las piezas codificadas.

In [3]:
data = pd.read_csv('Montabilidad_to_Quantum.csv',delimiter=';', encoding='latin-1')

columnas_a_quitar = ['file_type','metadata_path', 'metadata_date', 'metadata_size', 'query_key', 'text', 'Expediente', 'Código', 'N_Produccion', 'Fecha_Tecnica', 'metadata_file']
data = data.drop(columnas_a_quitar, axis=1)

data_denominacion = pd.read_excel('Codigos Denominacion 447.xlsx')

Se filtran en dos DataFrame diferentes los vectores Test y Train

In [3]:
def filtrado_vectores_train_test(data):
    vectores_train = data[data['Set'] == 'Train']
    vectores_test = data[data['Set'] == 'Test']
    return (vectores_test, vectores_train)

Función para normalizar un vector:  módulo = 1

In [4]:
def normalizar_vector(vector):
    norma = np.linalg.norm(vector)
    return vector/norma

Se le llama padding a la operación de rellenar un vector con 0s hasta llegar a un tamanño deseado.

En este caso necesitamos rellenar los vectores de tal forma que correspondan a al tamaño $2^n$ para poder codificarlos en estados cuánticos. Siendo n el numerpo de qbits utilizados.

In [5]:
def padding(vec, qubits):
    num_componentes_relleno = 2**qubits - len(vec)
    return np.pad(vec, (0, num_componentes_relleno), mode='constant', constant_values = 0)

Función que calcula el producto interno de dos estados cuánticos. 
> **input:** 2 vectores normalizados.

> **output:** producto interno. 0 si los vectores son totalmente diferentes 1 si son iguales.
1. Se comprueba que los vectores tengan el mismo tamaño, se calcula el numero de qbits necesarios para la operación y se aplica el padding a los vectores.
2. Se inicializan los estados cuánticos mediante la función initialize de Qiskit donde  los datos de los vectores se los codifica en amplitudes.
3. Se realizan las mediciones sobre la base canónica mediante la matriz Z de Pauli. Se procesa el resultado para obtener el producto interno.

La ejecución se hace sobre el simulador 'qasm_simulator'


In [6]:
def inner_prod(vec1, vec2):

    if len(vec1) != len(vec2):
        raise ValueError('Lengths of states are not equal')

    numero_componentes = len(vec1)
    nqubits = math.ceil(np.log2(numero_componentes))

    vec1 = padding(vec1, nqubits)
    vec2 = padding(vec2, nqubits)

    circ = QuantumCircuit(nqubits+1,1)
    vec = np.concatenate([vec1,vec2])/np.sqrt(2)

    circ.initialize(vec, range(nqubits+1))
    circ.h(nqubits)
    circ.measure(nqubits,0)

    backend = Aer.get_backend('qasm_simulator')
    job = execute(circ, backend, shots=20000)

    result = job.result()
    outputstate = result.get_counts(circ)

    if ('0' in outputstate.keys()):
        m_sum = float(outputstate["0"])/20000
    else:
        m_sum = 0

    inner_product = 2*m_sum-1
    return inner_product

In [7]:
def correlacion_vectores(vector_1, vector_2):
    test_normalizado = normalizar_vector(vector_1.iloc[ :-4].to_numpy())
    train_normalizado = normalizar_vector(vector_2.iloc[ :-4].to_numpy())
    return inner_prod(test_normalizado, train_normalizado)

In [8]:
def itera_por_matriz_cuantica(vectores_test, vectores_train):
  for index_test, test in vectores_test.iterrows():
    for index_train, train in vectores_train.iterrows():
      correlacion_entre_variables = correlacion_vectores(test, train)
      yield [f'test_{index_test}', f'train_{index_train}', correlacion_entre_variables, test['Categoria'], test['N_Pedido'],train['N_Pedido']]

Funciones para obtener la difernecia de componentes entre el vector Test exótico y el Train similar

In [9]:
def diferencias_componentes(exotico, similar):
    exotico = exotico.apply(lambda x: 1 if x != 0 else 0)
    similar = similar.apply(lambda x: 1 if x != 0 else 0)

    # Obtener las columnas que son 1 en `exotico` y 0 en `similar`
    condicion1 = (exotico == 1) & (similar == 0)
    diferencias_exotico = exotico[condicion1].index.tolist()

    # Obtener las columnas que son 1 en `similar` y 0 en `exotico`
    condicion2 = (similar == 1) & (exotico == 0)
    diferencias_similar = similar[condicion2].index.tolist()
    
    return diferencias_exotico,diferencias_similar

In [10]:
def obtener_denominacion(comp, data_denominacion):
    # Verifica si hay coincidencias
    if comp in data_denominacion['Codigo'].values:
        denominacion = data_denominacion[data_denominacion['Codigo'] == comp]['Denominación'].values[0]
    else:
        denominacion = 'No ha habido coincidencias'
    return denominacion

Función de creación del la tabla final

In [11]:
def creacion_resultado_final( exotico, similar, diferencia_exotico, diferencia_similar):
    # Crear el DataFrame
    columnas = [
        'Exotico',
        'Tipo_Vehículo',
        'Similar',
        'Descripción',
        'Componentes',
        'Id',
        'Categoria_exótico',
        'Categoria_similar',
        'Denominación'
    ]

    data = []

    # Iterar sobre la lista de diferencia exotico
    for comp in diferencia_exotico:
        entrada = [
            exotico['N_Pedido'],
            exotico['Tipo_vehículo'],
            similar['N_Pedido'],
            'Componentes en el exotico que no aparecen en el similar',
            'Mas',
            comp,
            exotico['Categoria'],
            similar['Categoria'],
            obtener_denominacion(comp, data_denominacion)
        ]
        data.append(entrada)

    # Iterar sobre la lista de diferencia similar
    for comp in diferencia_similar:
        entrada = [
            exotico['N_Pedido'],
            exotico['Tipo_vehículo'],
            similar['N_Pedido'],
            'Componentes en el similar que no aparecen en el exotico',
            'Menos',
            comp,
            exotico['Categoria'],
            similar['Categoria'],
            obtener_denominacion(comp, data_denominacion)
        ]
        data.append(entrada)

    # Crear el DataFrame final
    return pd.DataFrame(data, columns=columnas)

In [12]:
def resultados_csv(ruta_guardado, resultado, nombre_resultado):
    # Comprobar si la ruta de guardado existe, si no, crearla
    if not os.path.exists(ruta_guardado):
        os.makedirs(ruta_guardado)

    # Unir la ruta de guardado con el nombre del archivo y la extensión .csv
    ruta_completa = os.path.join(ruta_guardado, f"{nombre_resultado}.csv")

    # Guardar el DataFrame en el archivo CSV
    resultado.to_csv(ruta_completa, index=False)


In [13]:
ruta_guardado = 'D:\Project\I3B\Q4R - Mercedes\Resultados'

columnas = ['Nombre_Test', 'Nombre_Train', 'Correlacion', 'Categoria', 'N_Pedido_test', 'N_Pedido_train']
categorias = data.groupby('Categoria')

for nom_categoria, categoria in categorias:
    if nom_categoria != 'CLASE V':
         vectores_test, vectores_train = filtrado_vectores_train_test(categoria) 
         
         resultados_correlacion = pd.DataFrame(itera_por_matriz_cuantica(vectores_test, vectores_train), columns=columnas)
         resultados_correlacion_ordenado = resultados_correlacion.sort_values(by='Correlacion')

         

         exotico = pd.Series(vectores_test[vectores_test['N_Pedido'] == resultados_correlacion_ordenado.iloc[0]['N_Pedido_test']].iloc[0])
         
         similar = resultados_correlacion[resultados_correlacion['N_Pedido_test'] == exotico['N_Pedido']].sort_values(by='Correlacion').iloc[-1]
         train_similar = pd.Series(vectores_train[similar['N_Pedido_train'] == vectores_train['N_Pedido']].iloc[0])
         
         diferencias_exotico, diferencias_similar = diferencias_componentes(exotico.iloc[ :-4], train_similar.iloc[ :-4])

         resultado = creacion_resultado_final(exotico, train_similar, diferencias_exotico, diferencias_similar)

         nombre_resultado = f'resultados_{nom_categoria}'
         resultados_csv(ruta_guardado,resultado,nombre_resultado)
         
         display(resultados_correlacion_ordenado)

,Nombre_Test,Nombre_Train,Correlacion,Categoria,N_Pedido_test,N_Pedido_train
3333,test_683,train_53,0.0705,AVANTGARDE,5329280040,5129280241
3334,test_683,train_54,0.0756,AVANTGARDE,5329280040,5129280242
3135,test_677,train_36,0.0871,AVANTGARDE,5-30032056,5129280171
3337,test_683,train_90,0.1055,AVANTGARDE,5329280040,5129235042
3347,test_683,train_162,0.1057,AVANTGARDE,5329280040,5129280211
...,...,...,...,...,...,...
938,test_621,train_417,0.8854,AVANTGARDE,5-30033400,5229235349
2078,test_633,train_417,0.8857,AVANTGARDE,5-30033412,5229235349
1793,test_630,train_417,0.8874,AVANTGARDE,5-30033409,5229235349
1223,test_624,train_417,0.8897,AVANTGARDE,5-30033403,5229235349


,Nombre_Test,Nombre_Train,Correlacion,Categoria,N_Pedido_test,N_Pedido_train
642,test_666,train_247,0.2480,AVANTGARDE LINE ELECTRIC,5329233118,5129280304
702,test_684,train_277,0.2632,AVANTGARDE LINE ELECTRIC,5482300012,5129235097
698,test_684,train_232,0.2658,AVANTGARDE LINE ELECTRIC,5482300012,5129235092
697,test_684,train_230,0.2836,AVANTGARDE LINE ELECTRIC,5482300012,5129235089
723,test_684,train_447,0.2890,AVANTGARDE LINE ELECTRIC,5482300012,5329280028
...,...,...,...,...,...,...
567,test_615,train_404,0.6822,AVANTGARDE LINE ELECTRIC,5-30033364,5229235336
625,test_617,train_404,0.6836,AVANTGARDE LINE ELECTRIC,5-30033366,5229235336
538,test_614,train_404,0.6839,AVANTGARDE LINE ELECTRIC,5-30033363,5229235336
422,test_610,train_404,0.6840,AVANTGARDE LINE ELECTRIC,5-30033359,5229235336


,Nombre_Test,Nombre_Train,Correlacion,Categoria,N_Pedido_test,N_Pedido_train
0,test_489,train_1,0.1572,BASE,5-30033330,5129280232
62,test_491,train_1,0.1739,BASE,5-30033332,5129280232
31,test_490,train_1,0.1752,BASE,5-30033331,5129280232
32,test_490,train_21,0.2124,BASE,5-30033331,5129280197
1,test_489,train_21,0.2169,BASE,5-30033330,5129280197
...,...,...,...,...,...,...
154,test_524,train_367,0.9284,BASE,5-30033334,5329280047
123,test_523,train_367,0.9297,BASE,5-30033333,5329280047
91,test_491,train_352,0.9698,BASE,5-30033332,5229235300
29,test_489,train_352,0.9703,BASE,5-30033330,5229235300


,Nombre_Test,Nombre_Train,Correlacion,Categoria,N_Pedido_test,N_Pedido_train
42,test_585,train_276,0.3188,EXCLUSIVE,5-30032060,5129235096
22,test_584,train_276,0.3214,EXCLUSIVE,5-30032059,5129235096
182,test_680,train_276,0.3232,EXCLUSIVE,5-30032065,5129235096
62,test_586,train_276,0.3299,EXCLUSIVE,5-30032061,5129235096
141,test_665,train_167,0.3335,EXCLUSIVE,5329233114,5129280323
...,...,...,...,...,...,...
5,test_583,train_333,0.7114,EXCLUSIVE,5-30032058,5329280032
105,test_588,train_333,0.7118,EXCLUSIVE,5-30032063,5329280032
86,test_587,train_335,0.7118,EXCLUSIVE,5-30032062,5329280036
85,test_587,train_333,0.7149,EXCLUSIVE,5-30032062,5329280032


,Nombre_Test,Nombre_Train,Correlacion,Categoria,N_Pedido_test,N_Pedido_train
5,test_641,train_278,0.4091,MARCO POLO,5-30033422,5129235098
22,test_641,train_444,0.4499,MARCO POLO,5-30033422,5229235387
2,test_641,train_237,0.5158,MARCO POLO,5-30033422,5129280251
8,test_641,train_325,0.5279,MARCO POLO,5-30033422,5329280007
21,test_641,train_436,0.5295,MARCO POLO,5-30033422,5229235370
7,test_641,train_324,0.5374,MARCO POLO,5-30033422,5329280006
4,test_641,train_263,0.5411,MARCO POLO,5-30033422,5229280096
6,test_641,train_279,0.5466,MARCO POLO,5-30033422,5129280258
10,test_641,train_401,0.5470,MARCO POLO,5-30033422,5229235299
1,test_641,train_236,0.5472,MARCO POLO,5-30033422,5129280250


...

,Nombre_Test,Nombre_Train,Correlacion,Categoria,N_Pedido_test,N_Pedido_train
53,test_510,train_365,0.3013,SELECT,5329233109,5229235318
52,test_510,train_364,0.3043,SELECT,5329233109,5229235317
50,test_510,train_360,0.3073,SELECT,5329233109,5229235313
51,test_510,train_363,0.3096,SELECT,5329233109,5229235316
9,test_480,train_368,0.3969,SELECT,5-30033316,5353760499
...,...,...,...,...,...,...
65,test_520,train_360,0.9531,SELECT,5-30033327,5229235313
70,test_521,train_360,0.9550,SELECT,5-30033328,5229235313
60,test_519,train_360,0.9551,SELECT,5-30033326,5229235313
55,test_518,train_360,0.9573,SELECT,5-30033325,5229235313


,Nombre_Test,Nombre_Train,Correlacion,Categoria,N_Pedido_test,N_Pedido_train
1122,test_682,train_7,0.1147,STYLE,5-30032073,5129280177
1124,test_682,train_14,0.1206,STYLE,5-30032073,5129280199
1089,test_681,train_7,0.1208,STYLE,5-30032072,5129280177
1091,test_681,train_14,0.1242,STYLE,5-30032072,5129280199
35,test_590,train_14,0.1277,STYLE,5-30032067,5129280199
...,...,...,...,...,...,...
1020,test_673,train_452,0.9007,STYLE,5329280053,5329280050
1086,test_675,train_452,0.9010,STYLE,5329280055,5329280050
1051,test_674,train_450,0.9013,STYLE,5329280054,5329280048
1085,test_675,train_451,0.9017,STYLE,5329280055,5329280049


,Nombre_Test,Nombre_Train,Correlacion,Categoria,N_Pedido_test,N_Pedido_train
495,test_555,train_272,0.0841,TOURER PRO,5329233116,5129280356
615,test_565,train_272,0.0973,TOURER PRO,5-30033339,5129280356
528,test_560,train_340,0.1013,TOURER PRO,5482300009,5229280118
585,test_564,train_272,0.1030,TOURER PRO,5-30033338,5129280356
0,test_527,train_136,0.1115,TOURER PRO,5-30032074,5229280043
...,...,...,...,...,...,...
472,test_550,train_374,0.9797,TOURER PRO,5-30033396,5229235322
171,test_540,train_373,0.9797,TOURER PRO,5-30033386,5229235321
411,test_548,train_373,0.9799,TOURER PRO,5-30033394,5229235321
172,test_540,train_374,0.9804,TOURER PRO,5-30033386,5229235322


,Nombre_Test,Nombre_Train,Correlacion,Categoria,N_Pedido_test,N_Pedido_train
111,test_551,train_314,0.1775,TOURER SELECT,5229233110,5329280015
106,test_551,train_145,0.2083,TOURER SELECT,5229233110,5129280322
51,test_530,train_314,0.2453,TOURER SELECT,5-30032078,5329280015
110,test_551,train_310,0.2484,TOURER SELECT,5229233110,5329280030
32,test_529,train_147,0.2500,TOURER SELECT,5-30032077,5229280044
...,...,...,...,...,...,...
68,test_534,train_377,0.9549,TOURER SELECT,5-30033340,5229235326
98,test_536,train_377,0.9552,TOURER SELECT,5-30033342,5229235326
84,test_535,train_378,0.9552,TOURER SELECT,5-30033341,5229235327
85,test_535,train_379,0.9563,TOURER SELECT,5-30033341,5229235328


,Nombre_Test,Nombre_Train,Correlacion,Categoria,N_Pedido_test,N_Pedido_train
8,test_552,train_268,0.2683,TOURIER,5229235295,5129280259
6,test_552,train_152,0.2705,TOURIER,5229235295,5129280209
20,test_568,train_268,0.3411,TOURIER,5-30033348,5129280259
32,test_569,train_268,0.3471,TOURIER,5-30033349,5129280259
18,test_568,train_152,0.3487,TOURIER,5-30033348,5129280209
30,test_569,train_152,0.3573,TOURIER,5-30033349,5129280209
4,test_552,train_144,0.3589,TOURIER,5229235295,5129280311
1,test_552,train_83,0.3851,TOURIER,5229235295,5129280326
3,test_552,train_139,0.3892,TOURIER,5229235295,5129235111
16,test_568,train_144,0.4053,TOURIER,5-30033348,5129280311
